In this notebook, I discuss the various ways you can load and save data to/from SAM.

In [1]:
from samalg import SAM
import pandas as pd

/home/alec/anaconda3/envs/scrna/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


We can use `sam.load_data` to load data from files (namely `.csv`, `.txt`, and `.h5ad` files). For `.csv` and `.txt` files, the `sep` argument is the delimeter used in those files (usually `,` for `.csv` and `\t` for `.txt`). `sep=','` by default.

In [2]:
sam=SAM()
sam.__dict__

{'run_args': {}, 'preprocess_args': {}}

`sam.__dict__` shows all the current attributes of the SAM object. We can see that the SAM object is currently empty.

In [3]:
sam.load_data('../../example_data/schisto2.5_tpm.csv.gz' , sep = ',')
sam.__dict__

{'run_args': {},
 'preprocess_args': {},
 'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp'}

Now, we see that the SAM object is populated with the `AnnData` objects. `adata_raw` is the loaded data and will remain untouched. `adata` is initially set equal to `adata_raw` and will be subject to preprocessing and store the results of the SAM analysis.

When loading a dense expression table like from a `.csv` or a `.txt` file, you can use the `save_sparse_file` argument to save a sparse representation of the data (in the `AnnData` file format `.h5ad`) for much faster loading in the future. See:

In [4]:
sam=SAM()
sam.load_data('../../example_data/schisto2.5_tpm.csv.gz' , sep = ',', save_sparse_file = '../../example_data/sparse_data.h5ad')

In [5]:
sam=SAM()
sam.load_data('../../example_data/sparse_data.h5ad')
sam.__dict__

{'run_args': {},
 'preprocess_args': {},
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp',
 'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 }

We can also pass data directly into the SAM constructor via the `counts` argument. We can pass in an `AnnData` object:

In [6]:
example_anndata = sam.adata_raw
sam=SAM(counts = example_anndata)
sam.__dict__

{'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp',
 'run_args': {},
 'preprocess_args': {}}

We can also pass in a Pandas DataFrame:

In [7]:
DATAFRAME = pd.read_csv('../../example_data/schisto2.5_tpm.csv.gz',sep=',',index_col=0).T
sam=SAM(counts = DATAFRAME)
sam.__dict__

{'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp',
 'run_args': {},
 'preprocess_args': {}}

We can also pass in a tuple (scipy.sparse matrix, list of gene IDs, list of cell IDs):

In [8]:
sparse_data = sam.adata_raw.X
genes = sam.adata_raw.var_names
cells = sam.adata_raw.obs_names

sam=SAM(counts = (sparse_data,genes,cells))
sam.__dict__

{'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp',
 'run_args': {},
 'preprocess_args': {}}

We can also pass in the tuple (numpy array, list of gene IDs, list of cell IDs):

In [9]:
sparse_data = sparse_data.toarray()

sam=SAM(counts = (sparse_data,genes,cells))
sam.__dict__

{'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     layers: 'X_disp',
 'run_args': {},
 'preprocess_args': {}}

To save the SAM object, we can write the AnnData `sam.adata` to a '.h5ad' file. The '.h5ad' file will also contain the raw data in `sam.adata_raw`:

In [10]:
sam.preprocess_data()
sam.run()
sam.save_anndata('../../example_data/sam_results.h5ad')
sam.__dict__

RUNNING SAM
Iteration: 0, Convergence: 0.4606575406579718
Iteration: 1, Convergence: 0.13544171329818674
Iteration: 2, Convergence: 0.07162995525614586
Iteration: 3, Convergence: 0.021007430911942678
Iteration: 4, Convergence: 0.007672543686987294
Computing the UMAP embedding...
Elapsed time: 4.886494874954224 seconds


{'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 ,
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     var: 'mask_genes', 'spatial_dispersions', 'weights'
     uns: 'preprocess_args', 'ranked_genes', 'nnm', 'neighbors', 'run_args'
     obsm: 'X_pca', 'X_umap'
     layers: 'X_disp', 'X_knn_avg',
 'run_args': {'max_iter': 10,
  'verbose': True,
  'projection': 'umap',
  'stopping_condition': 0.005,
  'num_norm_avg': 50,
  'k': 20,
  'distance': 'correlation',
  'preprocessing': 'Normalizer',
  'npcs': None,
  'n_genes': None,
  'weight_PCs': True,
  'proj_kwargs': {}},
 'preprocess_args': {'div': 1,
  'sum_norm': None,
  'norm': 'log',
  'min_expression': 1,
  'thresh': 0.01,
  'filter_genes': True},
 'pca_obj': PCA(copy=True, iterated_power='auto', n_components=338, random_state=None,
     svd_solver='full', tol=0.0, whiten=False),
 'X_processed': (array([[0.        , 0.00313167, 0.        , ..., 0.00296594, 0.00404917,
          0.00413541],
         [0.     

Now, we can initialize an empty SAM object,

In [11]:
sam=SAM()
sam.__dict__

{'run_args': {}, 'preprocess_args': {}}

and use the `load_data` function to load the data back into the SAM object:

In [12]:
sam.load_data('../../example_data/sam_results.h5ad')
sam.__dict__

{'run_args': {},
 'preprocess_args': {},
 'adata': AnnData object with n_obs × n_vars = 338 × 10782 
     var: 'mask_genes', 'spatial_dispersions', 'weights'
     uns: 'neighbors', 'nnm', 'preprocess_args', 'ranked_genes', 'run_args'
     obsm: 'X_pca', 'X_umap'
     layers: 'X_disp', 'X_knn_avg',
 'adata_raw': AnnData object with n_obs × n_vars = 338 × 10782 }